In [2]:
import pandas as pd
import numpy as np

In [1]:
product_types = """
Проезд в трамвае, поездка
Проезд в троллейбусе, поездка
Проезд в метро, поездка
Проезд в городском автобусе,поездка
Полет в салоне экономического класса самолета, в расчете на 1000 км пути
Водоснабжение горячее, м3
Отопление, Гкал
Водоотведение, м3
Газ сжиженный, м3
Проживание в гостинице 1* или в мотеле, сутки с человека
Проживание в гостинице 2* , сутки с человека
Проживание в гостинице 3* , сутки с человека
Проживание в гостинице 4*-5* , сутки с человека
Проживание в хостеле, сутки с человека
Услуги по снабжению электроэнергией
Санаторий, день
Дом отдыха, пансионат, день
""".split("\n")

categories = """
Общественный транспорт
Общественный транспорт
Общественный транспорт
Общественный транспорт
Общественный транспорт
Коммунальные услуги
Коммунальные услуги
Коммунальные услуги
Коммунальные услуги
Места размещения
Места размещения
Места размещения
Места размещения
Места размещения
Коммунальные услуги
Места размещения
Места размещения
""".split("\n")

product_types = [item for item in product_types if len(item) > 0]
categories = [item for item in categories if len(item) > 0]


In [3]:
product_types_dict = {product_types[i]: categories[i] for i in range(len(categories))}

product_types_dict

{'Проезд в трамвае, поездка': 'Общественный транспорт',
 'Проезд в троллейбусе, поездка': 'Общественный транспорт',
 'Проезд в метро, поездка': 'Общественный транспорт',
 'Проезд в городском автобусе,поездка': 'Общественный транспорт',
 'Полет в салоне экономического класса самолета, в расчете на 1000 км пути': 'Общественный транспорт',
 'Водоснабжение горячее, м3': 'Коммунальные услуги',
 'Отопление, Гкал': 'Коммунальные услуги',
 'Водоотведение, м3': 'Коммунальные услуги',
 'Газ сжиженный, м3': 'Коммунальные услуги',
 'Проживание в гостинице 1* или в мотеле, сутки с человека': 'Места размещения',
 'Проживание в гостинице 2* , сутки с человека': 'Места размещения',
 'Проживание в гостинице 3* , сутки с человека': 'Места размещения',
 'Проживание в гостинице 4*-5* , сутки с человека': 'Места размещения',
 'Проживание в хостеле, сутки с человека': 'Места размещения',
 'Услуги по снабжению электроэнергией': 'Коммунальные услуги',
 'Санаторий, день': 'Места размещения',
 'Дом отдыха, панс

In [17]:
dataset = pd.read_excel("./data/Исходник_Для_парсера_УслугиСредние_потребительские_цены_на_услуги.xlsx")

In [18]:
area_coords = []

for i in range(len(dataset.columns)):
	item = dataset.iloc[2, i]
	item = str(item)
	if len(item) > 5:
		if len(area_coords) > 0:
			area_coords[-1].append([2, i])
		area_coords.append(
			[
				item,
				[2, i]
			]
		)
area_coords[-1].append([2, len(dataset.columns)])

area_coords.insert(0,
	['', [2, 1], [2, area_coords[0][1][-1]]]
)

In [19]:
area_coords

[['', [2, 1], [2, 5]],
 ['Центральный федеральный округ', [2, 5], [2, 81]],
 ['Северо-Западный федеральный округ', [2, 81], [2, 129]],
 ['Южный федеральный округ', [2, 129], [2, 165]],
 ['Северо-Кавказский федеральный округ', [2, 165], [2, 197]],
 ['Приволжский федеральный округ', [2, 197], [2, 257]],
 ['Уральский федеральный округ', [2, 257], [2, 285]],
 ['Сибирский федеральный округ', [2, 285], [2, 329]],
 ['Дальневосточный федеральный округ', [2, 329], [2, 377]]]

In [20]:
product_types_set = list(set(categories))
product_types_set

['Общественный транспорт', 'Коммунальные услуги', 'Места размещения']

In [31]:
new_dataset_columns = [
	"Дата",
	"Страна",
	"Наименование округа",
	"Наименование субъекта",
	"Тип агрегации",
	"Категория товара или услуги",
	"Наименование товара",
	"Средняя цена",
	"Позиция в рейтинге",

	"Общественный транспорт",
	"Коммунальные услуги",
	"Места размещения",
]

flat_dataset = {name: [] for name in new_dataset_columns}

def my_argsort(my_list, add=1):
	sorted_list = sorted(my_list)
	return [sorted_list.index(item)+add for item in my_list]

area_name = ""
for i in range(len(area_coords)):
	start_coords = area_coords[i]
	start_y, start_x = start_coords[1]
	end_y, end_x = start_coords[2]

	subject_name = dataset.iloc[start_y, start_x]
	for col in range(start_x, end_x):
		area_prices = {prod_type: [] for prod_type in product_types_set} 

		for j in range(7, len(dataset)-1):
			product_name = dataset.iloc[j, 0].strip()
			
			if len(str(dataset.iloc[3, col])) > 0 and not dataset.iloc[3, col] is np.nan:
				area_name = dataset.iloc[3, col]
			
			avg_price = dataset.iloc[j, col]
			date = dataset.iloc[6, col]
			products_type = product_types_dict[product_name]
			flat_dataset["Дата"].append(date)
			flat_dataset["Страна"].append("Российская Федерация")
			flat_dataset["Наименование округа"].append(subject_name)
			flat_dataset["Наименование субъекта"].append(area_name)
			flat_dataset["Категория товара или услуги"].append(products_type)
			flat_dataset["Наименование товара"].append(product_name)
			flat_dataset["Средняя цена"].append(avg_price)
			flat_dataset["Позиция в рейтинге"].append("")

			if products_type == "Общественный транспорт":
				flat_dataset['Общественный транспорт'].append("Общественный транспорт")
				flat_dataset['Коммунальные услуги'].append("")
				flat_dataset['Места размещения'].append("")

			if subject_name.strip() == "":
				flat_dataset["Тип агрегации"].append("Общее значение по РФ")
			
			if subject_name.strip() != "" and area_name.strip() == "":
				flat_dataset["Тип агрегации"].append("Общее значение по округу")

			if subject_name.strip() != "" and area_name.strip() != "":
				flat_dataset["Тип агрегации"].append("Общее значение по региону")


			if products_type == "Коммунальные услуги":
				flat_dataset['Общественный транспорт'].append("")
				flat_dataset['Коммунальные услуги'].append("Коммунальные услуги")
				flat_dataset['Места размещения'].append("")

			if products_type == "Места размещения":
				flat_dataset['Общественный транспорт'].append("")
				flat_dataset['Коммунальные услуги'].append("")
				flat_dataset['Места размещения'].append("Места размещения")

			# area_prices.append(avg_price)
			area_prices[products_type].append([len(flat_dataset["Позиция в рейтинге"]) - 1, avg_price])

		for product_type in product_types_set:
			prices_list = area_prices[product_type]
			prices_list_avg = [item[1] for item in prices_list]
			prices_list_avg = my_argsort(prices_list_avg)
			for k in range(len(prices_list)):
				pos = prices_list[k][0]
				position = prices_list_avg[k]
				flat_dataset["Позиция в рейтинге"][pos] = position


In [35]:
pd.DataFrame(data=flat_dataset).to_excel("./test.xlsx", index=False)